# Install and import required libraries

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import google.colab
from tqdm import tqdm
import re
import warnings
import csv

from flair.datasets import UD_INDONESIAN
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
warnings.filterwarnings("ignore")

# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/ruhutsitompul/done_labelled_ruhutsitompul.csv', delimiter=',', error_bad_lines=False)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,4178,1118666888647602177,2019-04-18 00:05:44,1118666888647602177,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...,@NediSetiadi Kami sedih krn biar sudah kalian ...,NaN,NaN,...,14,April,1,0,0,1,0.0,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...
4179,4179,1118665503642247170,2019-04-18 00:00:14,1118665503642247170,7,0,@TanYoana haiya Amoy ini makin stresssssss kac...,@TanYoana haiya Amoy ini makin stresssssss kac...,NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,@TanYoana haiya Amoy ini makin stresssssss kac...
4180,4180,1118664923301572608,2019-04-17 23:57:56,1118664923301572608,1684,200,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,NaN,NaN,...,31,April,0,0,0,0,0.0,0,0,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...
4181,4181,1118396091148460032,2019-04-17 06:09:41,1118396091148460032,6,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s...","@panca66 Ha ha ha maaf memilih diri sendiri, s...",NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s..."


In [ ]:
# Remove tweets without any aspect
df_tweets = df_tweets.loc[(df_tweets[['social', 'historical', 'dehumanization', 'accusation', 'attack', 'loyalty', 'threat']] != 0).any(axis=1)]
df_tweets = df_tweets.reset_index(drop=True)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,4151,1120875995962101760,2019-04-24 02:23:56,1120875995962101760,26,4,"@ustadtengkuzul Belajarlah menerima kekalahan,...","@ustadtengkuzul Belajarlah menerima kekalahan,...",NaN,NaN,...,12,April,1,0,0,1,0.0,0,0,"@ustadtengkuzul Belajarlah menerima kekalahan,..."
1660,4153,1120574450410217472,2019-04-23 06:25:42,1120574450410217472,1660,264,Aneh tapi Nyata ini bukan HOAX di Uk...,Aneh tapi Nyata ini bukan HOAX di Uk...,NaN,NaN,...,13,April,0,0,0,1,0.0,0,0,Aneh tapi Nyata ini bukan HOAX di Uk...
1661,4166,1119465028040527872,2019-04-20 04:57:15,1119465028040527872,1311,193,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...","Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...",NaN,NaN,...,24,April,1,0,0,1,0.0,0,0,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr..."
1662,4168,1119429279811428353,2019-04-20 02:35:12,1119429279811428353,10,0,@FerdinandHutah2 Masih saja kau menggonggong s...,@FerdinandHutah2 Masih saja kau menggonggong s...,NaN,NaN,...,6,April,0,0,1,0,0.0,0,0,@FerdinandHutah2 Masih saja kau menggonggong s...


# POS tagging with flair

In [ ]:
# 1. get the corpus
corpus = UD_INDONESIAN().downsample(0.1)
print(corpus)

# 2. what label do we want to predict?
label_type = 'upos'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-08 10:11:08,791 Reading data from /root/.flair/datasets/ud_indonesian
2023-03-08 10:11:08,793 Train: /root/.flair/datasets/ud_indonesian/id_gsd-ud-train.conllu
2023-03-08 10:11:08,795 Dev: /root/.flair/datasets/ud_indonesian/id_gsd-ud-dev.conllu
2023-03-08 10:11:08,799 Test: /root/.flair/datasets/ud_indonesian/id_gsd-ud-test.conllu
2023-03-08 10:11:24,973 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-08 10:11:25,889 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-08 10:11:28,366 Warning: An empty Sentence was created! Are there empty strings in your dataset?
Corpus: 448 train + 56 dev + 56 test sentences
2023-03-08 10:11:28,392 Computing label dictionary. Progress:


448it [00:00, 6848.97it/s]

2023-03-08 10:11:28,541 Dictionary created for label 'upos' with 17 values: NOUN (seen 2099 times), PROPN (seen 1768 times), PUNCT (seen 1326 times), VERB (seen 1041 times), ADP (seen 897 times), PRON (seen 538 times), ADJ (seen 392 times), NUM (seen 307 times), CCONJ (seen 302 times), ADV (seen 291 times), DET (seen 287 times), SCONJ (seen 190 times), AUX (seen 189 times), PART (seen 89 times), X (seen 7 times), SYM (seen 2 times)
Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, CCONJ, ADV, DET, SCONJ, AUX, PART, X, SYM


2023-03-08 10:12:07,736 SequenceTagger predicts: Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, CCONJ, ADV, DET, SCONJ, AUX, PART, X, SYM
2023-03-08 10:12:07,797 ----------------------------------------------------------------------------------------------------
2023-03-08 10:12:07,800 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'id-crawl'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): WordEmbeddings(
      'id'
      (embedding): Embedding(300686, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=600, out_features=600, bias=True)
  (rnn): LSTM(600, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=19, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-03-08 10:12:07,802 --------------------------------------------------------------

100%|██████████| 2/2 [00:00<00:00,  5.04it/s]

...

2023-03-08 10:15:06,404 ----------------------------------------------------------------------------------------------------
2023-03-08 10:15:13,552 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, CCONJ, ADV, DET, SCONJ, AUX, PART, X, SYM, <START>, <STOP>


100%|██████████| 2/2 [00:00<00:00,  4.31it/s]

2023-03-08 10:15:15,107 Evaluating as a multi-label problem: False


2023-03-08 10:15:15,142 0.7973	0.7973	0.7973	0.7973
2023-03-08 10:15:15,144 
Results:
- F-score (micro) 0.7973
- F-score (macro) 0.6173
- Accuracy 0.7973

By class:
              precision    recall  f1-score   support

        NOUN     0.6593    0.9330    0.7726       224
       PROPN     0.9437    0.7701    0.8481       261
        VERB     0.7843    0.9231    0.8481       130
       PUNCT     0.9929    1.0000    0.9964       139
         ADP     0.8571    0.9027    0.8793       113
        PRON     0.9038    0.9792    0.9400        48
         ADV     0.3846    0.3409    0.3614        44
         DET     0.8000    0.5217    0.6316        46
         ADJ     0.6818    0.2885    0.4054        52
       CCONJ     0.9333    0.8750    0.9032        32
         NUM     0.8519    0.8214    0.8364        28
         AUX     0.2778    0.2174    0.2439        23
       SCONJ     0.5385    0.3333    0.4118        21
        PART     1.0000    0.1000    0.1818        10
         SYM     0.0000 

{'test_score': 0.797274275979557,
 'dev_score_history': [0.29632408102025504,
  0.42460615153788445,
  0.5678919729932483,
  0.6241560390097525,
  0.6721680420105026,
  0.7051762940735183,
  0.7389347336834209,
  0.7591897974493623,
  0.7824456114028507,
  0.795198799699925],
 'train_loss_history': [2.5863374136407455,
  2.0340061279924484,
  1.6599990836881426,
  1.393549691717545,
  1.2258456930832262,
  1.0848034071738433,
  0.961092068002892,
  0.8911650465938303,
  0.8332131667737789,
  0.7772904783901028],
 'dev_loss_history': [2.2457034587860107,
  1.7855215072631836,
  1.4564083814620972,
  1.2352898120880127,
  1.0920048952102661,
  0.95794278383255,
  0.821689248085022,
  0.7707288861274719,
  0.6795819997787476,
  0.6341026425361633]}

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

tag_pos = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

with open('postagged.csv', mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for index, tweet in df_tweets['clean_tweets'].iteritems():
        sentence = Sentence(tweet)
        tag_pos.predict(sentence)
        writer.writerow([index, tweet, sentence.to_tagged_string()])

2023-03-08 10:15:21,212 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, CCONJ, ADV, DET, SCONJ, AUX, PART, X, SYM, <START>, <STOP>


#1-gram

In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in df_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')


2023-03-08 10:15:43,480 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, CCONJ, ADV, DET, SCONJ, AUX, PART, X, SYM, <START>, <STOP>
Top 10 most frequent POS tag patterns:
NOUN: 12854
PROPN: 4505
VERB: 2520
ADJ: 684
ADP: 436
PRON: 317
ADV: 299
PUNCT: 269
DET: 99
NUM: 53


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# specify the category for which to extract the most frequent POS tag patterns
social_tweets = df_tweets.loc[df_tweets.social== 1]

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in social_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns for Social aspect:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')

2023-03-08 10:16:06,316 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, CCONJ, ADV, DET, SCONJ, AUX, PART, X, SYM, <START>, <STOP>
Top 10 most frequent POS tag patterns for Social aspect:
NOUN: 6604
PROPN: 2173
VERB: 1307
ADJ: 328
ADP: 194
ADV: 139
PUNCT: 84
PRON: 79
DET: 56
NUM: 28


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# specify the category for which to extract the most frequent POS tag patterns
historical_tweets = df_tweets.loc[df_tweets.historical== 1]

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in historical_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns for Historical aspect:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')

2023-03-08 10:16:19,406 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, CCONJ, ADV, DET, SCONJ, AUX, PART, X, SYM, <START>, <STOP>
Top 10 most frequent POS tag patterns for Historical aspect:
NOUN: 303
PROPN: 121
VERB: 61
ADJ: 14
ADP: 10
DET: 4
PUNCT: 3
ADV: 3
PRON: 2
